### Fine-tuning DistilBERT for Text Classification

In [15]:
# libraries
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from datasets import Dataset

import nltk
from nltk.corpus import stopwords

In [9]:
# load dataset
df = pd.read_csv('train.csv')
df

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me to the 50 clásicos playlist,AddToPlaylist
2,i give this current textbook a rating value of...,RateBook
3,play the song little robin redbreast,PlayMusic
4,please add iris dement to my playlist this is ...,AddToPlaylist
...,...,...
13079,i want to eat choucroute at a brasserie for 8,BookRestaurant
13080,play funky heavy bluesy,PlayMusic
13081,rate the current album 2 points out of 6,RateBook
13082,go to the photograph the inflated tear,SearchCreativeWork


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13084 entries, 0 to 13083
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    13084 non-null  object
 1   intent  13084 non-null  object
dtypes: object(2)
memory usage: 204.6+ KB


In [11]:
df['intent'].value_counts()

intent
PlayMusic               1914
GetWeather              1896
BookRestaurant          1881
RateBook                1876
SearchScreeningEvent    1852
SearchCreativeWork      1847
AddToPlaylist           1818
Name: count, dtype: int64

In [12]:
df.isnull().sum()

text      0
intent    0
dtype: int64

In [14]:
# eda
df['intent'] = df['intent'].str.lower()
df['text'] = df['text'].str.lower()
df

,text,intent
0,listen to westbam alumb allergic on google music,playmusic
1,add step to me to the 50 clásicos playlist,addtoplaylist
2,i give this current textbook a rating value of...,ratebook
3,play the song little robin redbreast,playmusic
4,please add iris dement to my playlist this is ...,addtoplaylist
...,...,...
13079,i want to eat choucroute at a brasserie for 8,bookrestaurant
13080,play funky heavy bluesy,playmusic
13081,rate the current album 2 points out of 6,ratebook
13082,go to the photograph the inflated tear,searchcreativework


In [16]:
# prepare stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
# function to remove stopwords
def remove_stopwords(text):
  # text to lowercase
  text = text.lower()
  
  # remove stopwords
  filtered_words = ' '.join([word for word in text.split() if word not in stop_words])
  return filtered_words

  